This notebook contains model development code. I have used only 2 out of 5 tsv files to develop the solution in interest of time. My attempt while solving this assignment is to show the breadth of my knowledge and not necessarily build the best model. The current state of model development process is highly empirical and generally model development process takes multiple iterations to arrive at a solution. The general workflow in the notebook is as follows: 


1. Data Preprocessing/Feature Engineering/Data Cleaning
    * Processed/Filtered invalid/null values
    * Removed special characters 

* Feature Engineering
    
* Model Selection
    * Linear Regression
    * Decision Tree
    * Random Forest
    * XGBoost
    
* Model Evaluation 
    * MSE
    * RMSE
    * MAE
    
* Hyperparameter Tuning

To choose the best solution, I will follow following strategy:

- I will iterate multiple times through EDA, Data Preprocessing, Feature Engineering and Data Cleaning (in general it takes approx 70-80% of the time) to find optimal steps. 
- Assuming no other business metric dictates, I will evaluate multiple models on metrics like RMSE and MAE.
- Time to run the algorithm and other production related constraints also play important role in choosing the best model.

In [43]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import seaborn as sns
from scipy import stats

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import  RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

# Data Ingestion

In [42]:
# set random state for reproducing the results later
RANDOM_STATE=101

In [5]:
%%time

ratings=pd.read_csv("title.ratings.tsv", sep="\t")
title_basics=pd.read_csv("title.basics.tsv", sep="\t")

CPU times: user 14.3 s, sys: 2.14 s, total: 16.4 s
Wall time: 16.9 s


In [6]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1846
1,tt0000002,6.0,239
2,tt0000003,6.5,1612
3,tt0000004,6.0,155
4,tt0000005,6.2,2435


In [8]:
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [10]:
%%time

ratings_combined=ratings.merge(title_basics, on="tconst", how="inner")
ratings_combined.head()

CPU times: user 8.46 s, sys: 1.31 s, total: 9.78 s
Wall time: 9.78 s


,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,5.7,1846,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,6.0,239,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,6.5,1612,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,6.0,155,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,6.2,2435,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [11]:
# verify the shape of data after merge
assert ratings.shape[0]==ratings_combined.shape[0]

In [12]:
# rename columns
ratings_combined.columns=['tconst', 'average_rating', 'num_votes', 'title_type', 
                                'primary_title', 'original_title', 'is_adult', 'start_year', 
                                'end_year', 'runtime_minutes', 'genres']

In [14]:
ratings_movie=ratings_combined[ratings_combined.title_type=="movie"]
ratings_movie.drop(["end_year"], axis="columns", inplace=True)

ratings_movie.replace({"\\N":np.nan}, inplace=True)
ratings_movie.replace({None:np.nan}, inplace=True)
ratings_movie["is_adult"]=ratings_movie["is_adult"].astype(float).astype(str)
numeric_features=["average_rating", "num_votes", "start_year", "runtime_minutes"]
ratings_movie[numeric_features]=ratings_movie[numeric_features].astype(float)

ratings_movie=ratings_movie[ratings_movie.num_votes>10]

# Feature Engineering-Iteration 1

In [ ]:
# Create new feature for years since release
current_year = datetime.datetime.now().year
ratings_movie["years_since_release"]=current_year-ratings_movie["start_year"]

In [30]:
# creating a backup copy of data
backup=ratings_movie.copy()

In [31]:
non_req_cols=["tconst", "title_type", "original_title", "primary_title", "start_year", "genres"]
ratings_movie_non_req=ratings_movie[non_req_cols]
# drop features which won't go into model
ratings_movie.drop(non_req_cols, axis="columns", inplace=True)
# create numeric and categorical feature list
num_features=["years_since_release", "num_votes", "runtime_minutes"]
cat_features=["is_adult"]
target="average_rating"

# Train/Test Split

In [32]:
X=ratings_movie.drop([target], axis="columns")
y=ratings_movie[target]

assert ratings_movie.shape[0]==X.shape[0]
assert ratings_movie.shape[0]==y.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_STATE) 

assert X.shape[0]==X_train.shape[0]+X_val.shape[0]+X_test.shape[0]
assert y.shape[0]==y_train.shape[0]+y_val.shape[0]+y_test.shape[0]

It is common practice to fix the shape of distribution while developing with linear regression models. Most of the times, I choose between one of the following transformations empircally:
- Log
- Squareroot
- Reciprocal
- Box-cox

Note: Box-cox can be only applied to strictly positive values.

# Model Development

While choosing a model in this situation, I will first create a baseline model using a simpler model like Linear Regression or a Decision Tree and then move to more complex models, if required based on performance against a set of objective metrics and other constraints like runtime and maintainability of the model in production environment. 

* I will choose one of the following models a baseline:
    * Logistic Regression
    * Decision Tree
    * SVM
    * Random Forest
    
* More Advanced models:
    * AdaBoost
    * XGBoost
    * Word2Vec
    * Neural Network with Embedding layers
    

- I also tend to prefer tree-based models for baseline because they require less preprocessing/cleaning. As they are robust to handle missing values and outliers, so it is quicker to build a prototype using these models.
- One rule of thumb for putting any model in production is __simpler models__ are easier to maintain. 

## Candidate 1-Linear Regression

In [33]:
def build_pipeline(num_features, cat_features):
    numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", PowerTransformer())])
    categorical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="constant", fill_value="missing")), ("ohe", OneHotEncoder(handle_unknown="ignore"))])
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_features),
            ("cat", categorical_transformer, cat_features),
        ])
    return preprocessor
preprocessor=build_pipeline(num_features, cat_features)    

pipe_linear = Pipeline(
    steps=[("preprocessor", preprocessor), ("linear_reg", LinearRegression())]
)
pipe_linear.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   PowerTransformer())]),
                                                  ['years_since_release',
                                                   'num_votes',
                                                   'runtime_minutes']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
           

# Model Evaluation


In [34]:
def evaluate(model, X, y, dataset_type="validation"):
    y_pred=model.predict(X)
    rmse=mean_squared_error(y, y_pred)**(0.5)
    print(f"data: {dataset_type} RMSE: {rmse}")
    # R2 score
    r2=r2_score(y, y_pred)
    print(f"data: {dataset_type} R-Squared: {r2}")
    return rmse, r2
y_val_pred=evaluate(pipe_linear, X_val, y_val, dataset_type="validation")
y_train_pred=evaluate(pipe_linear, X_train, y_train, dataset_type="train")

data: validation RMSE: 1.2952682610299573
data: validation R-Squared: 0.005674568897768628
data: train RMSE: 1.3015973476970262
data: train R-Squared: 0.005686135817215221


In [ ]:
lin_residuals=y_val-y_val_pred
plt.scatter(lin_residuals,y_val_pred)
plt.show()

The model has almost no predictive power right now. The empirical nature of model development process requires us to go back to data and extract or develop new features and iterate multiple times to improve. This step is called Feature Engineering.

# Feature Engineering-Iteration 2

This part of the job is more art than science and this will provide maximum improvement in the model performance. I will now add one more feature: `Genres` into the model. 

In [37]:
ratings_movie=backup.copy()

def remove_space(x):
    if isinstance(x, str):
        return str.lower(x.replace(" ", ""))
    else:
        return np.nan 
def process_genre(data):
    temp=data.copy()
    temp['genres'] = temp['genres'].apply(remove_space)
    temp=pd.concat([temp, temp.genres.str.split(",", expand=True)], axis="columns")
    temp.rename(columns={0:"genre_0", 1: "genre_1", 2: "genre_2"}, inplace=True)
    temp.drop(["genres"], axis="columns", inplace=True)
    return temp

ratings_movie=process_genre(ratings_movie)   
non_req_cols=["tconst", "title_type", "original_title", "primary_title", "start_year"]
ratings_movie_non_req=ratings_movie[non_req_cols]
ratings_movie.drop(non_req_cols, axis="columns", inplace=True)
num_features=["years_since_release", "num_votes", "runtime_minutes"]
cat_features=["is_adult", "genre_0", "genre_1", "genre_2"]
target="average_rating"
ratings_movie.head()

,average_rating,num_votes,is_adult,runtime_minutes,years_since_release,genre_0,genre_1,genre_2
340,4.5,14.0,0.0,100.0,117.0,NaN,NaN,NaN
374,6.1,743.0,0.0,70.0,116.0,action,adventure,biography
383,5.2,16.0,0.0,90.0,115.0,drama,None,None
397,4.5,23.0,0.0,NaN,115.0,drama,None,None
405,3.8,24.0,0.0,NaN,114.0,drama,None,None


In [38]:
X=ratings_movie.drop([target], axis="columns")
y=ratings_movie[target]

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_STATE) 

# create pipeline
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)
categorical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="constant", fill_value="missing")), ("ohe", OneHotEncoder(handle_unknown="ignore"))]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features),
    ]
)
pipe_linear = Pipeline(
    steps=[("preprocessor", preprocessor), ("linear_reg", LinearRegression())]
)
# fit model 
pipe_linear.fit(X_train, y_train)
# evaulate model
y_val_pred=evaluate(pipe_linear, X_val, y_val, dataset_type="validation")
y_train_pred=evaluate(pipe_linear, X_train, y_train, dataset_type="train")

data: validation RMSE: 1.1561394985256819
data: validation R-Squared: 0.20780959658523002
data: train RMSE: 1.159858835283928
data: train R-Squared: 0.21044851388292163


## Candidate 2-DecisionTree

As explained before, the tree-based models can handle outliers and missing values automatically so the pipeline steps for treating them first is not required here. However, in interest of time I will use the same pipeline for all my models.

In [39]:
pipe_dt = Pipeline(
    steps=[("preprocessor", preprocessor), ("decision_tr", DecisionTreeRegressor(max_depth=4, random_state=RANDOM_STATE))]
)
pipe_dt.fit(X_train, y_train)
y_val_pred_dt=evaluate(pipe_dt, X_val, y_val, dataset_type="validation")
y_train_pred_dt=evaluate(pipe_dt, X_train, y_train, dataset_type="train")


data: validation RMSE: 1.1868704869022317
data: validation R-Squared: 0.16513594889437966
data: train RMSE: 1.1909044480738376
data: train R-Squared: 0.1676154322913187


## Candidate 3-Random Forest

In [40]:
pipe_rf = Pipeline(
    steps=[("preprocessor", preprocessor), ("rf", RandomForestRegressor(max_depth=4, random_state=RANDOM_STATE))]
)
pipe_rf.fit(X_train, y_train)
y_val_pred_rf=evaluate(pipe_rf, X_val, y_val, dataset_type="validation")
y_train_pred_rf=evaluate(pipe_rf, X_train, y_train, dataset_type="train")

data: validation RMSE: 1.1848407096018967
data: validation R-Squared: 0.16798906401072067
data: train RMSE: 1.1892742030973331
data: train R-Squared: 0.16989279713087146


## Candidate 4-XGBoost

In [41]:
pipe_xgb = Pipeline(
    steps=[("preprocessor", preprocessor), ("xgb", XGBRegressor(n_estimators=100, max_depth=6, eta=0.1, subsample=0.7, colsample_bytree=0.8, random_state=RANDOM_STATE))]
)
pipe_xgb.fit(X_train, y_train)
y_val_pred_xgb=evaluate(pipe_xgb, X_val, y_val, dataset_type="validation")
y_train_pred_xgb=evaluate(pipe_xgb, X_train, y_train, dataset_type="train")

[02:36:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
data: validation RMSE: 1.0834717592060452
data: validation R-Squared: 0.30426429579303604
data: train RMSE: 1.0770345393717178
data: train R-Squared: 0.3191844808602571


# Hyperparameter Optimization using Randomized Search
 
For the sake of this excercise and demostrating my familiarity with the concept, I will go ahead and use RandomizedSearch for XGBoost model. 

I generally prefer RandomizedSearch over GridSearch because empirically Randomized Search has proven to converge faster. In recent years, Bayesian hyperparameter search has become very popular and is faster than both above options. Bayesian Hyperparameter Search have some strong open source ([scikit-optimize](https://scikit-optimize.github.io/stable/)) and proprietary solutions like [AWS Sagemaker Hyperparameter Tuning](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) module. I have used AWS SageMaker's bayesian hyperparameter optimization module extensively in my past assignments.

* In general, the guiding principle will be __Bias Variance Trade-Off__ which means you turn the knobs between complicated model and simple model using these hyperparameters. We say that a model has a high bias if it is not able to fully use the information in the data. We say that a model has high variance if it is using too much information from the the data. More complexity is associated with higher variance.

* In multiple iterations, I would have performed hyperparameter search by making educated guesses in each consecutive iteration following the logic mentioned below.

* If model is performing better on training data as compared to test data, we have problem of __Overfitting__ and we can do following with hyperparameters:
    * Decrease max_depth
    * Decrease num_round
    * Increase min_child_weight
    * Increase gamma
    * Increase alpha (L1 regularization)
    * Increase lambda (L2 regularization)
    * Decrease colsample_bytree
    * Decrease subsample
        
* If model is performing bad on both training data and test data, we have problem of __Underfitting__ and we can create a more complex model by following the exact opposite direction for hyperparameter values as mentioned above.

### The following codeblock is running out of memory on my local machine but this can run on a EC2 or any distributed environment with more compute like SageMaker

In [ ]:
%%time

# Adding more parameters will give better exploring power but will increase processing time in a combinatorial way
params = {'learning_rate' :stats.uniform(0.001,0.2),
          'n_estimators':[10,50,100, 500, 1000, 2000],
          'gamma':stats.uniform(0,0.02),
          'subsample':(0.2,0.3,0.4,0.5,0.6, 0.7, 0.8),
          'reg_alpha':[25,50,75,100,150,200],
          'reg_lambda':[25,50,75,100,150,200],
          'max_depth':np.arange(1,11),
          'min_child_weight':np.arange(1,11)}

pipe_xgb = Pipeline(
    steps=[("preprocessor", preprocessor), ("xgb", XGBRegressor(random_state=RANDOM_STATE))]
)
research_cv = RandomizedSearchCV(estimator=pipe_xgb, param_distributions=params, n_iter=15, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
research_cv.fit(X_train, y_train)
print("Best estimator: ",research_cv.best_estimator_)
print("Best Cross Validation Score: ",research_cv.best_score_)

# What Else?

Most of the lift in the model performance is done while iterating through data while preprocessing and cleaning and 5-10% uplift can be observed by using Hyperparameter optimization. 


# References

During this take-home assignment, I went through a lot of good online resources some of which I am including here for reference.

- Applied Machine Learning [AI for All Humans](https://cloud.google.com/blog/topics/developers-practitioners/ai-all-humans-course-delight-and-inspire?utm_campaign=6166fac5808ba700018472fd&utm_content=61af8d861b2ece00012a28e8&utm_medium=smarpshare&utm_source=linkedin)
- Text Extraction [O'Reilly Applied Text Analysis](https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/ch04.html)
- Sklearn randomized search cv [Randomized Search CV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
- Kaggle Movie Recommendations: [Kaggle Movie Recommendations](https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system)
